In [10]:
import gradio as gr
import numpy as np
import cv2
from skimage.color import rgb2gray

In [11]:
def transform_image(img):
  return rgb2gray(img)

In [12]:
demo = gr.Interface(fn = transform_image,
  inputs = gr.Image(),
  outputs = "image")

In [13]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5697d65b42c976a85f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

In [15]:
demo = gr.Interface(sepia, gr.Image(), "image", api_name="predict")

In [16]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f08c0d2a9a6d2b1b10.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Interactive Edge Detection (Multiple Inputs)
Complexity Upgrade: Adds interactive sliders (gr.Slider) to control algorithm parameters in real-time. This is useful for tuning computer vision filters.

In [17]:
def edge_detector(img, low_threshold, high_threshold):
    # Gradio provides the image as a generic numpy array (RGB)
    # We need to convert to grayscale for Canny
    if img is None:
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Apply Canny Edge Detection
    # Edges are white, background is black
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    return edges

In [18]:
demo = gr.Interface(
    fn=edge_detector,
    inputs=[
        gr.Image(label="Upload Image"),
        gr.Slider(0, 255, value=50, label="Low Threshold (Sensitivity)"),
        gr.Slider(0, 255, value=150, label="High Threshold (Hysteresis)")
    ],
    outputs=gr.Image(label="Detected Edges"),
    title="Interactive Edge Detection",
    description="Adjust the sliders to change how strict the edge detection is."
)

In [19]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e9afac1c9df16b7212.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Real-Time AI Classification (Deep Learning Backend)
Uses a pre-trained Deep Learning model (ResNet50) to classify the image. This demonstrates how to wrap a complex ML inference pipeline into a simple user interface.

In [20]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

In [24]:
# 1. Load a pre-trained ResNet50 model (trained on ImageNet)
model = ResNet50(weights='imagenet')

def classify_image(img):
    if img is None:
        return None

    # 2. Resize the image
    # ResNet expects exactly 224x224 pixels
    img_resized = tf.image.resize(img, (224, 224))

    # Convert the TF Tensor back to a NumPy array for easier processing
    img_array = img_resized.numpy()

    # 3. Format dimensions: (Height, Width, 3) -> (1, Height, Width, 3)
    # The model expects a "batch" of images, even if it's just one.
    img_array = np.expand_dims(img_array, axis=0)

    # 4. Preprocess (Normalize pixel values)
    # This aligns the image colors with what ResNet saw during training
    img_array = preprocess_input(img_array)

    # 5. Predict
    predictions = model.predict(img_array)

    # 6. Decode results
    # Returns the top 3 classes
    decoded = decode_predictions(predictions, top=3)[0]

    # 7. Format for Gradio
    # We convert numpy floats to python floats to avoid JSON errors
    return {label: float(score) for (_, label, score) in decoded}

In [25]:
demo = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="numpy", label="Upload Image"),
    outputs=gr.Label(num_top_classes=3, label="Predictions"),
    title="AI Image Classifier (ResNet50)",
    description="Upload an image to see what the AI thinks it is."
)

In [26]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2b0b86584e2e1dfa01.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
